```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Notebook description

This notebook is primarly to rapidly test the test simulations.
It can also be used to produce the "correct" results, which are used for comparison in the actual tests.

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')


import unittest
import numpy as np
import sys
import gc
from importlib import reload

from testUtils import *

sys.path.insert(0, '../')
from SWESimulators import Common
from SWESimulators import CDKLM16
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BathymetryAndICs as BC
from SWESimulators import DataAssimilationUtils as dautils
from SWESimulators import IEWPFOcean
from SWESimulators import IPythonMagic

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
## Make sim, ensemble and IEWPF instance

if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
if 'iewpfOcean' in globals():
    iewpfOcean.cleanUp()



reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)    
reload(IEWPFOcean)
    
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

waterDepth = 10.0

ensembleSize = 3
driftersPerOceanModel = 3

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
               nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
rel_grid_size = nx*1.0/dx
BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
eta0 = eta0*0.3
BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
eta0 = eta0*(-1.3)
BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
eta0 = eta0*1.0
BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
eta0 = eta0*(-0.03)
BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
eta0 = eta0 + 0.02*eta0_extra
BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, \
                                   f, beta, g, \
                                   nx, ny, dx, dy, ghosts)
eta0 = eta0*0.5

q0 = 0.5*dt*f/(g*waterDepth)

sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, \
                      g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)


ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(gpu_ctx, ensembleSize,sim,
                                                      num_drifters = driftersPerOceanModel,
                                                      observation_type=dautils.ObservationType.DirectUnderlyingFlow,
                                                      observation_variance = 0.01**2)

iewpfOcean = IEWPFOcean.IEWPFOcean(ensemble, show_errors=True)


In [ ]:
### Obtain initial stuff:

all_innovations = ensemble.getInnovations()
innovation = all_innovations[0]
target_weight = iewpfOcean.obtainTargetWeight(ensemble, all_innovations)


def run_ensemble(ensemble):
    return ensemble.step(1000*ensemble.dt)



In [ ]:

## Investigate in S = (HQH^T + R)^-1

S_ref = np.loadtxt("iewpfRefData/S.dat")
print(S_ref)

print ("(S_ref, iewpfOcean.S_host, diff)")
print ((S_ref))
print ((iewpfOcean.S_host))
print ((S_ref - iewpfOcean.S_host)/S_ref)

#np.savetxt("iewpfRefData/S.dat", iewpfOcean.S_host)


In [ ]:
## Investigate in the SVD result

ref_SVD = np.loadtxt("iewpfRefData/localSVD.dat")


fig = plt.figure(figsize=(4,4))
plt.imshow(iewpfOcean.localSVD_host, interpolation="None", origin='lower')
plt.colorbar()
plt.title("square root of SVD from class")

fig = plt.figure(figsize=(4,4))
plt.imshow(ref_SVD, interpolation="None", origin='lower')
plt.colorbar()
plt.title("square root from reference")

fig = plt.figure(figsize=(4,4))
plt.imshow(ref_SVD - iewpfOcean.localSVD_host, interpolation="None", origin='lower')
plt.colorbar()
plt.title("diff square roots")


full_from_class = np.dot(iewpfOcean.localSVD_host, iewpfOcean.localSVD_host.transpose())
full_from_ref   = np.dot(ref_SVD, ref_SVD.transpose())

fig = plt.figure(figsize=(4,4))
plt.imshow(full_from_class - np.eye(49), interpolation="None", origin='lower')
plt.colorbar()
plt.title("full SVD from class")

fig = plt.figure(figsize=(4,4))
plt.imshow(full_from_ref - np.eye(49), interpolation="None", origin='lower')
plt.colorbar()
plt.title("full SVD from reference")

fig = plt.figure(figsize=(4,4))
plt.imshow(full_from_ref - full_from_class, interpolation="None", origin='lower')
plt.colorbar()
plt.title("diff full")




#np.savetxt("iewpfRefData/localSVD.dat", iewpfOcean.localSVD_host)


In [ ]:
# Testing setNoiseBufferToZero
reload(IEWPFOcean)
iewpfOcean = IEWPFOcean.IEWPFOcean(ensemble, show_errors=True)

# Make random numbers
ensemble.particles[0].small_scale_model_error.generateNormalDistribution()
random_numbers = ensemble.particles[0].small_scale_model_error.getRandomNumbers()

fig = plt.figure(figsize=(4,4))
plt.imshow(random_numbers, interpolation="None", origin='lower')
plt.colorbar()
plt.title("Random numbers ")

# Set the random number buffer to zero
iewpfOcean.setNoiseBufferToZero(ensemble.particles[0])
random_numbers_zero = ensemble.particles[0].small_scale_model_error.getRandomNumbers()

fig = plt.figure(figsize=(4,4))
plt.imshow(random_numbers_zero, interpolation="None", origin='lower')
plt.colorbar()
plt.title("Random numbers zero")

sys.getrefcount(sim.gpu_stream)
